In [15]:
# small LSTM Network to Generate Text for Alice in Wonderland
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [13]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

 Total Characters:  147773
Total Vocab:  49


In [9]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  147673


In [10]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

print X.shape

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

(147673, 100, 1)


In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
147673/147673 [==============================] - 977s - loss: 2.9560   
Epoch 2/20
147673/147673 [==============================] - 974s - loss: 2.7466   
Epoch 3/20
147673/147673 [==============================] - 975s - loss: 2.6332   
Epoch 4/20
147673/147673 [==============================] - 973s - loss: 2.5485   
Epoch 5/20
147673/147673 [==============================] - 970s - loss: 2.4742   
Epoch 6/20
147673/147673 [==============================] - 977s - loss: 2.4168   
Epoch 7/20
147673/147673 [==============================] - 984s - loss: 2.3677   
Epoch 8/20
147673/147673 [==============================] - 980s - loss: 2.3262   
Epoch 9/20
147673/147673 [==============================] - 972s - loss: 2.2808   
Epoch 10/20
147673/147673 [==============================] - 971s - loss: 2.2447   
Epoch 11/20
147673/147673 [==============================] - 971s - loss: 2.2046   
Epoch 12/20
147673/147673 [==============================] - 975s - loss: 2.1746   
E

In [16]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
filename = "weights-improvement-19-1.9377.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" at?' thought alice to herself.

'of the mushroom,' said the caterpillar, just as if she had asked  "
and
berute an in sas oo toe thneg harten ano ohe to the tabbit saabit  soode and the tai oh the tabbit saseere the was toe tiieg her head the
hert an the could. 

'i whanl sas a little 'laseer,' said the kante 'int io a long tare to
an in the tai shan i 
and yhu  toene to the thet '

the dormouse sea whit so ho a lintle oo the sooee aeain, and thete
tas nor inre the was a little sooee an the cad bedn the wanted the 
aeren th the tas a little sooe  a tirtll of the tuoen, and then she whs
low lene the madt of the 
aoucdrse to tey at the cad teten hnr her  a finde of thing sare tiiee an
the rabte was oo the tabbit  aadute in 
the rast was i van g lent of 
thete shen io the same to tee
a aota                                                                                                                                                                                                  

In [17]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, nb_epoch=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
147673/147673 [==============================] - 2609s - loss: 2.8184  
Epoch 2/50
147673/147673 [==============================] - 3224s - loss: 2.4503  
Epoch 3/50
147673/147673 [==============================] - 2497s - loss: 2.2569  
Epoch 4/50
147673/147673 [==============================] - 2500s - loss: 2.1200  
Epoch 5/50
147673/147673 [==============================] - 2500s - loss: 2.0132  
Epoch 6/50
147673/147673 [==============================] - 2503s - loss: 1.9329  
Epoch 7/50
147673/147673 [==============================] - 2509s - loss: 1.8668  
Epoch 8/50
147673/147673 [==============================] - 2504s - loss: 1.8133  
Epoch 9/50
147673/147673 [==============================] - 2505s - loss: 1.7631  
Epoch 10/50
147673/147673 [==============================] - 2520s - loss: 1.7194  
Epoch 11/50
147673/147673 [==============================] - 2507s - loss: 1.6798  
Epoch 12/50
147673/147673 [==============================] - 2510s - loss: 1.6501  
E

In [19]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
filename = "weights-improvement-48-1.2319-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" goose, with the bones and the beak--
    pray how did you manage to do it?'

   'in my youth,' sa "
id the king, 'the marter was a little

now and alice fou so the table. but the was not a lomg tur of the had
lent the tank as the could gold in the court, and she thought the was

of the thoee of the thoee of the table. 'i don't know the door tay of
the sable about the sight wild the tiings bll the tenser on the temtence
on the tay the other uile the was all the temtenced the tame as the
cook of the court, and she thought the whole party she was sert hear

the was teriectly as she could got some time with the tempence of the
court, and she thought the whole party she was solething to see it out
the was to say the thme, and the whole parcerand she was to say the

it!s garden, and teem the would lane any rere
Done.
